# Utilisation du modèle

Importation des dependances

In [ ]:
import pickle

import cv2
import mediapipe as mp
import numpy as np

Chargement de la config

In [ ]:
import json

with open('config.json', 'r') as f:
   config = json.load(f)

MODEL_FILE = config['model']['file']
DATASET_DEFINITION = config['dataset']['definition']

Chargement du modele permettant de detecter les mains pour afficher les points sur les mains

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

Chargement du modele

In [ ]:
model_dict = pickle.load(open(MODEL_FILE, 'rb'))
model = model_dict['model']

Fonction permettant de récupérer les positions X et Y de chaque point de la main.

In [ ]:
def get_hand_data(multi_hand_landmarks):
    data = []
    for landmark in multi_hand_landmarks:
        data.append(landmark.x)
        data.append(landmark.y)
    return data

Fonction permettant d'afficher les points sur les mains

In [ ]:
def display_hands_points(frame, multi_hand_landmarks):
    if multi_hand_landmarks:
        for hand_landmarks in multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

Fonction permettant d'afficher la boîte autour de la main

In [ ]:
def display_hands_box(frame, title, H, W, data):
    x_ = data[::2]
    y_ = data[1::2]
    
    x1 = int(min(x_) * W)
    y1 = int(min(y_) * H)
    
    x2 = int(max(x_) * W)
    y2 = int(max(y_) * H)
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
    cv2.putText(frame, title, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3, cv2.LINE_AA)

Logique principal :

- Afficher les points de chaque main.
- Afficher des boites autour de chaque main avec la prédiction de notre modèle.

In [ ]:
cap = cv2.VideoCapture(0)

while True:    
    ret, frame = cap.read()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    
    display_hands_points(frame, results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        
        H, W, _ = frame.shape
        for hand_landmarks in results.multi_hand_landmarks:
            data = get_hand_data(hand_landmarks.landmark)
            
            prediction = model.predict([np.asarray(data)])
            prediction_name = DATASET_DEFINITION[prediction[0]]
            
            display_hands_box(frame, prediction_name, H, W, data)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)